In [1]:
from keras.models import load_model
from py2neo import Node, Relationship,Graph
from custorn import *
from config import *
import numpy as np
from sklearn.externals import joblib
import jieba
graph = Graph('http://localhost:7474',username='neo4j',password='13086573198zzz')


Using TensorFlow backend.
c:\python3.6\lib\site-packages\requests\__init__.py:80: RequestsDependencyWarning: urllib3 (1.24.3) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)
c:\python3.6\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
entity_model=load_model('.//model//model.h5',custom_objects={'Position_Embedding':Position_Embedding(128),'Attention':Attention(nb_head=8,size_per_head=128),
                                                            'ndim':ndim,'batch_dot':batch_dot})
relation_model=model=joblib.load('.//model//lr.model')
tfidf_model=joblib.load('.//model//tfidf.model')

Tensor("attention_1/Reshape_2:0", shape=(?, 150, 1024), dtype=float32)
Tensor("attention_1/transpose_1:0", shape=(?, 8, ?, 128), dtype=float32)
Tensor("attention_1/Reshape_7:0", shape=(?, ?, 8, 128), dtype=float32)
Tensor("attention_1/transpose_5:0", shape=(?, 8, ?, 128), dtype=float32)
Tensor("attention_1/transpose_1:0", shape=(?, 8, ?, 128), dtype=float32) Tensor("attention_1/transpose_3:0", shape=(?, 8, ?, 128), dtype=float32)
Tensor("attention_1/truediv:0", shape=(?, 8, ?, ?), dtype=float32)


In [3]:

def predict_entity(text):
    ids=np.reshape([word2int[word] for word in text]+[0]*(150-len(text)),(1,150))
    pred=np.argmax(np.squeeze(entity_model.predict(ids)),axis=-1)
    entity=""
    type_=""
    for i in range(len(pred)):
        if pred[i]!=0:
            entity+=text[i]
    for i in range(len(pred)):
        if pred[i]!=0:
            type_+=ids2tag[pred[i]].split('_')[1].split('_')[0]
            break
    return entity,type_
def predict_yitu(text):
    sparse=tfidf_model.transform([' '.join(jieba.lcut(text))])
    pred=relation_model.predict(sparse.toarray())[0]
    return pred
def remove_text(text,type_):
    for word in entity:
        pre_text=text.replace(entity,entity_relation_map[type_]).replace('《','').replace('》','').replace('<<','').replace('>>','')
    return pre_text
def find_ans(entity,type_,relation):
    entity=entity
    type_=type_
    relation=relation
    str='MATCH (obj:'+type_+' {name: "'+entity+'"})-[:'+relation+']->(sub) RETURN properties(sub)'
    str2='MATCH (obj)-[:'+relation+']->(sub:'+type_+' {name: "'+entity+'"}) RETURN properties(obj)'
    ans=list(graph.run(str))
    if len(ans)==0:
        ans=list(graph.run(str2))
    if len(ans)==0:
        return '不知道哦'
    else:
        names=[pp.values()[0]['name'] for pp in ans]
    return names

In [4]:
text='《无尽的爱》的主演是谁?'
entity,type_=predict_entity(text)
pre_text=remove_text(text,type_)
relation=predict_yitu(pre_text)
print(entity,type_,relation)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\夏鹤珑\AppData\Local\Temp\jieba.cache
Loading model cost 1.518 seconds.
Prefix dict has been built succesfully.


无尽的爱 影视作品 主演


In [5]:
find_ans(entity,type_,relation)

['马国贤', '林嘉俐', '洪流', '检场', '杨丽音']